In [13]:
import cv2
from ipywidgets import Video
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [14]:
video_path = 'Chess_Data\\chess_easy1.mp4'

In [15]:
video_capture = cv2.VideoCapture(video_path)

### Read one frame to calibrate camera

In [16]:
def save_first_frame(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Release the video capture object
    cap.release()

    return frame

first_frame = save_first_frame(video_path)

In [17]:
def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

In [18]:
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150)

frame_with_boxes = first_frame.copy()
dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
dst = cv2.dilate(dst, None)
corners = np.argwhere(dst > 0.01 * dst.max())

# Find corners closest to image corners
image_corners = np.array([[0, 0], [0, first_frame.shape[1]], [first_frame.shape[0], 0], [first_frame.shape[0], first_frame.shape[1]]])
closest_corners = find_closest_points_to_corners(corners, image_corners)

target_square_size = 1000  # Adjust the size as needed
target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)

### Full detection on easy/medium

In [19]:
# video_capture = cv2.VideoCapture(video_path)

# while True:
#     ret, frame = video_capture.read()
#     if not ret:
#         break
    
#     homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

#     # Apply the perspective transformation
#     calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

#     grid_img = calibrated_image.copy()
#     offset = 59 # Board edge offset
#     for x in np.linspace(offset, 1000-offset, 9, dtype=int):
#         cv2.line(grid_img, (x, 0+offset), (x, 1000-offset), color=(0, 0, 255), thickness=2)  
#         cv2.line(grid_img, (0+offset, x), (1000-offset, x), color=(0, 0, 255), thickness=2)  

#     # Display the result
#     cv2.imshow('Chessboard focus', grid_img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

## Hard detection

TODO: Solve the hand problem
- Check how big is the change in corner placement in new and last ***ACCEPTED*** frame and accept only ones that dont change too much

OR

- Remove "hand detected" frames

In [27]:
def get_squares_occupation(calibrated_image, grid_division, show_img=True):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.75
    thickness = 2

    square_size = grid_division[1] - grid_division[0]

    calibrated_gray = cv2.cvtColor(calibrated_image, cv2.COLOR_BGR2GRAY)
    calibrated_edges = cv2.Canny(calibrated_gray, 50, 150)

    space_img = calibrated_image.copy()

    empty_check_offset = 20
    kernel_size = square_size - empty_check_offset
    artifact_tolerance = 300
    col_imgs = []
    square_color = "Black"
    for x in grid_division[:-1]:
        row_imgs = []
        for y in grid_division[:-1]:
            x_d = x+empty_check_offset//2, x+empty_check_offset//2+kernel_size
            y_d = y+empty_check_offset//2, y+empty_check_offset//2+kernel_size

            edge_board_square = calibrated_edges[x_d[0]:x_d[1], y_d[0]:y_d[1]]
            edge_board_square_bin = np.where(edge_board_square==255, 1, edge_board_square)
            edge_board_square_value = np.sum(edge_board_square_bin)

            row_imgs.append(edge_board_square)
            # imshow(resize_img(edge_board_square, 1))

            if edge_board_square_value > artifact_tolerance:
                # cv2.putText(space_img, f"{square_color} sq", (y+20, x+50), font, font_scale, font_color, thickness)
                cv2.putText(space_img, f"Figure", (y+16, x+60), font, font_scale, (0, 0, 255), thickness)

            elif edge_board_square_value <= artifact_tolerance:
                # cv2.putText(space_img, f"{square_color} sq", (y+20, x+50), font, font_scale, font_color, thickness)
                cv2.putText(space_img, f"Empty", (y+16, x+60), font, font_scale, (0, 255, 0), thickness)
                
            square_color = "White" if square_color == "Black" else "Black"
                
        square_color = "White" if square_color == "Black" else "Black"

        # imshow(resize_img(np.concatenate(row_imgs, 1), 1))
        col_imgs.append(np.concatenate(row_imgs, 1))

    cv2.imshow('Labelled_img', space_img)


In [28]:
video_capture = cv2.VideoCapture(video_path)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)

    frame_with_boxes = frame.copy()
    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    # Find corners closest to image corners
    image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)

    target_square_size = 1000  # Adjust the size as needed
    target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    
    homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

    # Apply the perspective transformation
    calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

    grid_img = calibrated_image.copy()
    offset = 60
    grid_division = np.linspace(offset, 1000-offset, 9, dtype=int)
    for x in grid_division:
        cv2.line(grid_img, (x, 0+offset), (x, 1000-offset), color=(0, 0, 255), thickness=2)  
        cv2.line(grid_img, (0+offset, x), (1000-offset, x), color=(0, 0, 255), thickness=2)  
    
    # Get occupation
    get_squares_occupation(calibrated_image, grid_division)
    
    # cv2.imshow('Chessboard focus', calibrated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

# Not adjusted

In [18]:
# Util euclidean distance function
def get_corner_diff(ref_corners, new_corners):
    ref_corners = np.array(ref_corners)
    new_corners = np.array(new_corners)

    distances = np.linalg.norm(ref_corners - new_corners, axis=1)
    
    return sum(distances)

In [29]:
video_capture = cv2.VideoCapture(video_path)

is_first_frame = True
max_tolerance = 60
offset_frame_tolerance = max_tolerance
distance_threshold = 50
history_queue = []
while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)

    frame_with_boxes = frame.copy()
    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    # Find corners closest to image corners
    image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)

    target_square_size = 1000  # Adjust the size as needed
    target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    
    if is_first_frame:
        ref_corners = closest_corners.copy()
        is_first_frame = False

    # If change in corners is big, use last accepted reference corners
    print(get_corner_diff(ref_corners, closest_corners))
    if get_corner_diff(ref_corners, closest_corners) > distance_threshold:
        offset_frame_tolerance -= 1
        if offset_frame_tolerance == 0:
            offset_frame_tolerance = max_tolerance
            ref_corners = closest_corners.copy()
        
        homography_matrix, _ = cv2.findHomography(ref_corners[:,::-1], target_points)
    
    else:
        ref_corners = closest_corners.copy()
        homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

    # Apply the perspective transformation
    calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

    grid_img = calibrated_image.copy()
    offset = 59 # Board edge offset
    for x in np.linspace(offset, 1000-offset, 9, dtype=int):
        cv2.line(grid_img, (x, 0+offset), (x, 1000-offset), color=(0, 0, 255), thickness=2)  
        cv2.line(grid_img, (0+offset, x), (1000-offset, x), color=(0, 0, 255), thickness=2)  

    # Display the result
    cv2.imshow('Chessboard focus', grid_img)

    # Display the result
    # cv2.imshow('Chessboard focus', calibrated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

0.0
0.0
6.650281539872885
4.82842712474619
5.06449510224598
5.414213562373095
4.82842712474619
7.8863495173726745
0.0
4.414213562373095
6.242640687119286
1.0
2.414213562373095
3.0
1.0
4.414213562373095
0.0
3.0
4.16227766016838
5.242640687119286
5.123105625617661
0.0
0.0
6.708203932499369
0.0
5.16227766016838
6.06449510224598
4.414213562373095
5.414213562373095
6.06449510224598
6.23606797749979
9.123105625617661
7.650281539872885
5.0
6.576491222541475
5.650281539872885
3.23606797749979
3.0
1.4142135623730951
2.0
0.0
1.4142135623730951
1.0
4.414213562373095
6.06449510224598
3.6502815398728847
5.06449510224598
4.650281539872885
4.23606797749979
3.23606797749979
2.23606797749979
5.16227766016838
5.650281539872885
0.0
3.0
2.0
2.0
1.0
4.414213562373095
1.4142135623730951
2.414213562373095
5.242640687119285
3.414213562373095
6.06449510224598
5.242640687119286
3.23606797749979
4.650281539872885
3.414213562373095
2.0
3.0
6.8863495173726745
4.242640687119286
4.414213562373095
6.478708664619075
5

### Other techniques

In [22]:
video_capture = cv2.VideoCapture(video_path)

corners_queue = []
corners_queue_length = 70

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)

    frame_with_boxes = frame.copy()
    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    # Find corners closest to image corners
    image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)

    target_square_size = 1000  # Adjust the size as needed
    target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    
    if len(corners_queue) < corners_queue_length:
        corners_queue.append(closest_corners)
    else:
        corners_queue.pop(0)
        corners_queue.append(closest_corners)

    homography_matrix, _ = cv2.findHomography(np.median(corners_queue, axis=0)[:,::-1], target_points)
    
    # Apply the perspective transformation
    calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

    grid_img = calibrated_image.copy()
    offset = 59 # Board edge offset
    for x in np.linspace(offset, 1000-offset, 9, dtype=int):
        cv2.line(grid_img, (x, 0+offset), (x, 1000-offset), color=(0, 0, 255), thickness=2)  
        cv2.line(grid_img, (0+offset, x), (1000-offset, x), color=(0, 0, 255), thickness=2)  

    # Display the result
    cv2.imshow('Chessboard focus', grid_img)

    # Display the result
    # cv2.imshow('Chessboard focus', calibrated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()